In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)  # The dimensions will be reduced to (8, 8, 8) due to the pooling applied in the forward pass
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        return F.relu(self.conv2(out))

In [7]:
conv = CNN()
conv

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)